In [1]:
#GPU' ya bağlanma
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 155320 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.27-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [2]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [3]:
import os
os.chdir("/content/drive/Udemy/Titanic")
!pwd

/content/drive/Udemy/Titanic


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import re

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam 


In [16]:
def preprocess(data):
    
    #Kabin
    data.Cabin.fillna('0', inplace=True)
    data.loc[data.Cabin.str[0] == 'A', 'Cabin'] = 1
    data.loc[data.Cabin.str[0] == 'B', 'Cabin'] = 2
    data.loc[data.Cabin.str[0] == 'C', 'Cabin'] = 3
    data.loc[data.Cabin.str[0] == 'D', 'Cabin'] = 4
    data.loc[data.Cabin.str[0] == 'E', 'Cabin'] = 5
    data.loc[data.Cabin.str[0] == 'F', 'Cabin'] = 6
    data.loc[data.Cabin.str[0] == 'G', 'Cabin'] = 7
    data.loc[data.Cabin.str[0] == 'T', 'Cabin'] = 8
    
    # Cinsiyeti tam sayıya çevirelim
    data['Sex'].replace('female', 1, inplace=True)
    data['Sex'].replace('male', 2, inplace=True)
    
    # Gemiye biniş limanlarını tam sayıya çevirelim
    data['Embarked'].replace('S', 1, inplace=True)
    data['Embarked'].replace('C', 2, inplace=True)
    data['Embarked'].replace('Q', 3, inplace=True)
    
    # Olmayan (NA) yaş değerlerini medyan ile dolduralım
    data['Age'].fillna(data['Age'].median(), inplace=True)
    data['Fare'].fillna(data['Fare'].median(), inplace=True)
    data['Embarked'].fillna(data['Embarked'].median(), inplace=True)
    
    # İstersek olmayan (NA) değerleri tamamen silebiliriz
    # data.dropna(subset=['Fare', 'Embarked'], inplace=True, how='any')
    return data

In [18]:
def group_titles(data):
    data['Names'] = data['Name'].map(lambda x: len(re.split(' ', x)))
    data['Title'] = data['Name'].map(lambda x: re.search(', (.+?) ', x).group(1))
    data['Title'].replace('Master.', 0, inplace=True)
    data['Title'].replace('Mr.', 1, inplace=True)
    data['Title'].replace(['Ms.','Mlle.', 'Miss.'], 2, inplace=True)
    data['Title'].replace(['Mme.', 'Mrs.'], 3, inplace=True)
    data['Title'].replace(['Dona.', 'Lady.', 'the Countess.', 'Capt.', 'Col.', 'Don.', 'Dr.', 'Major.', 'Rev.', 'Sir.', 'Jonkheer.', 'the'], 4, inplace=True)


In [7]:
def data_subset(data):
    features = ['Pclass', 'SibSp', 'Parch', 'Sex', 'Names', 'Title', 'Age', 'Cabin'] #, 'Fare', 'Embarked']
    lengh_features = len(features)
    #öznitelik sayısı
    subset = data[features]#.fillna(0)
    return subset, lengh_features

Kullanacağımız model tipi sequential. Sequential, Keras’ta bir model oluşturmanın en kolay yoludur. Katman ile bir model katman oluşturmanıza izin verir. ‘Dense’, katman türüdür. Dense, çoğu durumda çalışan standart bir katman türüdür. Yoğun bir katmanda, önceki katmandaki tüm düğümler mevcut katmandaki düğümlere bağlanır. Epoch(döngü) sayısı, eğitim sırasında tüm eğitim verilerinin ağa gösterilme sayısıdır. Mini batch size(boyutu), parametre güncellemesinin gerçekleştiği ağa verilen alt örneklerin sayısıdır. SOFTPLUS aktivasyon fonksiyonudur.

In [20]:
from keras import callbacks
def create_model(train_set_size, input_length, num_epochs, batch_size):
    model = Sequential()
    model.add(Dense(7, input_dim=input_length, activation='softplus'))
    model.add(Dense(3, activation='softplus'))
    model.add(Dense(1, activation='softplus'))
#öğrenme oranı
    lr=.001
    adam0=Adam(lr=lr)

#modelin derlenmesi

     # Modeli derleyip ve daha iyi bir sonuç elde edildiğinde ağırlıkları kaydedelim
    model.compile(loss='binary_crossentropy', optimizer=adam0, metrics=['accuracy'])
   #en iyi ağırlıkları kaydedeceğimiz konum
    filepath = 'weights.best.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    history_model = model.fit(X_train[:train_set_size], Y_train[:train_set_size], callbacks=callbacks_list, epochs=num_epochs, batch_size=batch_size, verbose=0) #40, 32
    return model, history_model
#hesaplanan sonuçlar history_model de kayıtlı, bu bilgileri çizidrmek için aşağı daki işlemleri yapalğım
def plots(history):
  loss_history=history.history['loss']
  acc_history=history.history['accuracy']
  epochs=[(i+1) for i in range (num_epochs) ]

  ax=plt.subplot(211)
  ax.plot(epochs,loss_history,color='red')
  ax.set_xlabel('Epochs')
  ax.set_ylabel('Error Rate\n')
  ax.set_title('Error Rate per Epoch\n')

#splot: çizdireceğimiz grafikleri tek bir eksende çizdirmemizi sağlar.
  ax2 = plt.subplot(212)
  ax2.plot(epochs, acc_history, color='blue')
  ax2.set_xlabel('Epochs')
  ax2.set_ylabel('Accuracy\n')
  ax2.set_title('Accuracy per Epoch\n')

  plt.subplots_adjust(hspace=0.8)
  plt.savefig('Accuracy_loss.png')
  plt.close()

In [9]:
def test(batch_size):
  test=pd.read_csv('test.csv', header=0)
  test_ids=test['PassengerId']
  test=preprocess(test)
  group_titles(test)
  testdata, _=data_subset(test)

  X_test=np.array(testdata).astype(float)
  output=model.predict(X_test,batch_size=batch_size,verbose=0)
  output=output.reshape((418,))

#passengerId ile kişilerin hayattta kalıp kalmadığı bilgilerinin biribirne bağlanması

  column_1 = np.concatenate((['PassengerId'], test_ids ), axis=0 )
  column_2 = np.concatenate( ( ['Survived'], output ), axis=0 )

  f=open("output.csv","w")
  writer=csv.writer(f)
  for i in range(len(column_1)):
    writer.writerow([column_1[i]]+[column_2[i]])

  f.close()


Eğitim ve Test İşlemleri - Sonuçların Grafikleştirilmesi

In [21]:
#sonuçların yeniden üretilebilir olması ve eğitim kümesini okuma işlemleri
#eğitim kümesinde ön işlemlerin gerçekleştirilmesi
import numpy as np
import pandas as pd

seed = 7
np.random.seed(seed)


train = pd.read_csv('train.csv', header=0)


preprocess(train)
group_titles(train)


num_epochs = 100
batch_size = 32



traindata, lengh_features = data_subset(train)

Y_train = np.array(train['Survived']).astype(int)
X_train = np.array(traindata).astype(float)


train_set_size = int(.67 * len(X_train))


model, history_model = create_model(train_set_size, lengh_features, num_epochs, batch_size)

plots(history_model)


X_validation = X_train[train_set_size:]
Y_validation = Y_train[train_set_size:]


loss_and_metrics = model.evaluate(X_validation, Y_validation, batch_size=batch_size)
print ("loss_and_metrics")

test(batch_size)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)



Epoch 1: accuracy improved from -inf to 0.39094, saving model to weights.best.hdf5

Epoch 2: accuracy did not improve from 0.39094

Epoch 3: accuracy improved from 0.39094 to 0.40604, saving model to weights.best.hdf5

Epoch 4: accuracy improved from 0.40604 to 0.60738, saving model to weights.best.hdf5

Epoch 5: accuracy improved from 0.60738 to 0.63423, saving model to weights.best.hdf5

Epoch 6: accuracy improved from 0.63423 to 0.64597, saving model to weights.best.hdf5

Epoch 7: accuracy did not improve from 0.64597

Epoch 8: accuracy improved from 0.64597 to 0.66779, saving model to weights.best.hdf5

Epoch 9: accuracy improved from 0.66779 to 0.67282, saving model to weights.best.hdf5

Epoch 10: accuracy did not improve from 0.67282

Epoch 11: accuracy did not improve from 0.67282

Epoch 12: accuracy did not improve from 0.67282

Epoch 13: accuracy improved from 0.67282 to 0.67953, saving model to weights.best.hdf5

Epoch 14: accuracy improved from 0.67953 to 0.69966, saving mo